<a href="https://colab.research.google.com/github/TiagolGoulart/ScoreIbovTools/blob/main/listaNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''Esse código busca as últimas 10 notícias do dia relacionadas a bolsa de valores
e salva no banco de dados ibovStocks.db'''

import sqlite3
from googlesearch import search
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from urllib.parse import quote

# Abrir conexão com o arquivo .db
conn = sqlite3.connect('/content/drive/MyDrive/Score_Ibov_Project/ibovStocks.db')
c = conn.cursor()
# Deleta as notícias anteriores
c.execute("DELETE FROM News")

# Com base nas palavras chaves dentro de query e no tempo de 1 dia contando o momento atual
# monta a lista de descrições e links

query = "B3 Bolsa de valores IBOVESPA"
timeframe = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d %H:%M:%S') + " " + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
timeframe_url = quote(timeframe)
results = []

for j in search(query, num=10, stop=10,tbs=f'cdr:1,cd_min:{timeframe_url},cd_max:', lang='pt-BR', tpe='nws', pause=2):
    results.append(j)

titles = []
for link in results:
    try:
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser', from_encoding='utf-8')
        title = soup.find('title').get_text()
        titles.append(title)
    except:
        titles.append("Veja agora mesmo!")

i=1
for link, title in zip(results, titles):
    # Executa um comando SQL de inserção na tabela desejada

    c.execute("INSERT INTO News (Id, Description, URL) VALUES (?, ?, ?)", (str(i), title, link))
    i+=1
    # Confirma a inserção no banco de dados
    conn.commit()

# Fecha a conexão com o banco de dados
conn.close()